In [1]:
#Установка нужных пакетов
!pip install --upgrade nltk gensim bokeh umap-learn

import itertools
import string

import numpy as np
import umap
from nltk.tokenize import WordPunctTokenizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.3 MB/s 
     |████████████████████████████████| 18.5 MB 556 kB/s 
     |████████████████████████████████| 88 kB 7.0 MB/s 
     |████████████████████████████████| 1.1 MB 44.8 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=864820aa8825367c5c912937d50365453501d71d80394156af1dab90fbf6b323
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55509 sha256=d5b511d8e710d96a8823ca15ed1c181acb50a6e2a887e556863377ab8039c10e
  Stored in directory: /root/.cache/pip/wheels/19/bc/eb/974072a56a7082a302f8b4be1ad6d21bf5019235c2eff65928
Successfully built umap-learn pynndescent
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gen

In [34]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# выгружаем датасет:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc

--2022-11-20 09:31:36--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2022-11-20 09:31:36--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc27085705be842ad05d1b66b854.dl.dropboxusercontent.com/cd/0/get/BxEI4EeXPdzqzazQTA_cgp-fF8kpB97lXdB8vkcO4nza5ycW_4fNiAqmo5Ow7mgHzaiz3R9KMSvQkdIEe1FC5GOqN3Jx0T0mm-MmgtaG248uAlb2s4Of0kRvLL9W9ju3KN2fSlmlslbd329cqQqlVk9gw4kCcxQOoEsMlW3-4VW_kw/file?dl=1# [following]
--2022-11-20 09:31:36--  https://uc27085705be842ad05d1b66b854.dl.dropboxusercontent.com/cd/0/get/BxEI4EeXPdzqzazQTA_cgp-fF8kpB97lXdB8vkcO4nza5ycW_4fNiAqmo5Ow7mgHzaiz3R9KMSvQkdIEe1FC5GO

In [3]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

In [4]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


#Задание 1: Перевести все слова в нижний регистр (NLTK) из data и добавьте как лист токенов в листе data_tok


In [6]:
data_tok = []
for i in range(len(data)): 
  data_tok.append(tokenizer.tokenize(data[i].lower()))


#checking

assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"


###Задание 2: Подсчитайте топ10 самых популярных лем в рамках data

In [9]:
import pandas as pd
lemmatizer = WordNetLemmatizer()
dict_words = {}

for i in range(len(data_tok)):
  for key in data_tok[i]:
    if lemmatizer.lemmatize(key) in dict_words.keys():
      dict_words[lemmatizer.lemmatize(key)] += 1
    else:
      dict_words.update({lemmatizer.lemmatize(key): 1})



In [16]:
df = pd.DataFrame(dict_words.items())
df.sort_values(by=1,ascending=False).head(10)

,0,1
17,?,552413
30,the,252068
18,what,214798
10,is,185392
24,a,172513
1,i,149873
22,to,141788
37,in,139813
60,how,135687
49,of,112001


###Задание 3: Подсчитайте количество разных слов до и после лемматизации

До лемматизации

In [17]:
lem_words = []
for _ in data_tok:
    for value in _:
        lem_words.append(value)


In [21]:
before = pd.DataFrame(lem_words)
len(before[0].unique())


87819

После лемматизации

In [22]:
len(df)

80303

###Задание 4: Подсчитайте количество разных слов до и после стемминга

До стемминга

In [23]:
len(before[0].unique())

87819

После стемминга

In [36]:
stemm = PorterStemmer()
stem_words = {}
for _ in data_tok:
  for word in _: 
    stem_word = stemm.stem(word)
    if stem_word in stem_words.keys(): 
      stem_words[stem_word] += 1
    else:
      stem_words[stem_word] = 1

In [40]:
after = pd.DataFrame(stem_words.items())
len(after)

67026

###Задание 5: Подсчитайте количество разных слов



REGEXP

https://www.programiz.com/python-programming/regex 

https://docs.python.org/3/howto/regex.html

In [ ]:
import re

pattern = 'a*s'
test_string = 'abyss'
result = re.match(pattern, test_string)

if result:
  print("Search successful.")
else:
  print("Search unsuccessful.")	

Search unsuccessful.


###Задание 6: 
https://www.hackerrank.com/challenges/matching-specific-string/problem?isFullScreen=true 

###Задание 7: 
https://www.hackerrank.com/challenges/matching-whitespace-non-whitespace-character/problem?isFullScreen=true

###Задание 8: 
https://www.hackerrank.com/challenges/matching-start-end/problem?isFullScreen=true

###Задание 9: 
https://www.hackerrank.com/challenges/matching-word-boundaries/problem?isFullScreen=true

Bag Of Words (BOW)

In [ ]:
def vectorize(tokens):
    ''' This function takes list of words in a sentence as input 
    and returns a vector of size of filtered_vocab.It puts 0 if the 
    word is not present in tokens and count of token if present.'''
    vector=[]
    for w in filtered_vocab:
        vector.append(tokens.count(w))
    return vector
def unique(sequence):
    '''This functions returns a list in which the order remains 
    same and no item repeats.Using the set() function does not 
    preserve the original ordering,so i didnt use that instead'''
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

#create a list of stopwords.You can import stopwords from nltk too
stopwords=["to","is","a"]

#list of special characters.You can use regular expressions too
special_char=[",",":"," ",";",".","?"]

#Write the sentences in the corpus,in our case, just two 
string1="Welcome to Great Learning , Now start learning"
string2="Learning is a good practice"

#convert them to lower case
string1=string1.lower()
string2=string2.lower()

#split the sentences into tokens
tokens1=string1.split()
tokens2=string2.split()
print(tokens1)
print(tokens2)

#create a vocabulary list
vocab=unique(tokens1+tokens2)
print(vocab)

#filter the vocabulary list
filtered_vocab=[]
for w in vocab: 
    if w not in stopwords and w not in special_char: 
        filtered_vocab.append(w)
print(filtered_vocab)

#convert sentences into vectords
vector1=vectorize(tokens1)
print(vector1)
vector2=vectorize(tokens2)
print(vector2)

['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'learning']
['learning', 'is', 'a', 'good', 'practice']
['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'is', 'a', 'good', 'practice']
['welcome', 'great', 'learning', 'now', 'start', 'good', 'practice']
[1, 1, 2, 1, 1, 0, 0]
[0, 0, 1, 0, 0, 1, 1]


Задание 10: Реализовать Bag of words на data_tok (можно на NLTK, можно без)